# Test scrapping 

In [1]:
from dotenv import load_dotenv
import os
import openrouteservice
from openrouteservice import convert
import folium
import geopy
from geopy.geocoders import Nominatim
import time



In [2]:
load_dotenv()
api_key = os.getenv('openroute_api_key')


In [3]:
client = openrouteservice.Client(key=api_key) # Specify your personal API key


In [ ]:
coords = ((8.34234,48.23424),(8.34423,48.26424))
routes = client.directions(coords)

print(routes)

In [ ]:
coords = ((8.34234,48.23424),(8.34423,48.26424), (8.34523,48.24424), (8.41423,48.21424),(8.34234,48.23424))

routes = client.directions(coords, profile='cycling-regular', optimize_waypoints=True)

print(routes)

In [ ]:
routes['routes'][0]['geometry']

In [7]:
geometry=routes['routes'][0]['geometry']
decoded = convert.decode_polyline(geometry)


In [ ]:
decoded['coordinates'][:10]

In [ ]:
m= folium.Map(location = list(reversed(routes['bbox'])),zoom_start=13)


In [ ]:
map_center = [(routes['bbox'][0] + routes['bbox'][2]) / 2, (routes['bbox'][1] + routes['bbox'][3]) / 2]

#m= folium.Map(location = map_center,zoom_start=13)
m= folium.Map(location = map_center,zoom_start=12)
for i, coord in enumerate(coords):
    folium.Marker(
        location=coord,
        popup=f"Point {i+1}: {coord}",
        icon=folium.Icon(color="red", icon="info-sign")
    ).add_to(m)
folium.PolyLine(locations =  decoded['coordinates'],color='blue').add_to(m)
m

## Test Geocodage

In [ ]:

def geocode_addresses(addresses,city):
    """
    Géocode une liste d'adresses en utilisant l'API OpenRouteService.

    :param addresses: Liste d'adresses (chaînes de caractères) à géocoder.
    :param api_key: Clé API OpenRouteService.
    :return: Liste de tuples (adresse, latitude, longitude).
    """
    results = []
    
    for address in addresses:
        try:
            # Appeler l'API pour obtenir les coordonnées
            response = client.pelias_search(text=f"{address},{city}")
            if response and 'features' in response and response['features']:
                # Extraire les coordonnées (longitude, latitude)
                coordinates = response['features'][0]['geometry']['coordinates']
                results.append((coordinates))  # Latitude, Longitude
            else:
                results.append((address, None, None))  # Aucune correspondance trouvée
        except Exception as e:
            print(f"Erreur lors du géocodage de l'adresse '{address}': {e}")
            results.append((address, None, None))
    
    return results




In [ ]:
tab=[]
for adresse in list_act:
    full_adresse = f"{adresse},{ville}"
    response = client.pelias_search(full_adresse)
    tab.append(list(reversed(response['features'][0]['geometry']['coordinates'])))

In [16]:

def geocode_addresses_2(addresses,city):
    """
    Géocode une liste d'adresses en utilisant l'API OpenRouteService.

    :param addresses: Liste d'adresses (chaînes de caractères) à géocoder.
    :param api_key: Clé API OpenRouteService.
    :return: Liste de tuples (adresse, latitude, longitude).
    """
    results = {}
    geolocator = Nominatim(user_agent="test")

    for address in addresses:
        try:
            # Appeler l'API pour obtenir les coordonnées
            location = geolocator.geocode(f"{address},{city}")
            if location:
                results[address] = [location.longitude,location.latitude]  # Latitude, Longitude
            
        except Exception as e:
            print(f"Erreur lors du géocodage de l'adresse '{address}': {e}")
    
    return results





In [ ]:
lieux = geocode_addresses_2(list_act,ville)



In [ ]:
coords

In [ ]:
cod = tuple(tuple(coord) for coord in lieux.values())
cod

In [ ]:
routes = client.directions(list(lieux.values()), profile='cycling-regular', optimize_waypoints=True)


In [ ]:
print(routes['routes'][0]['summary']['distance']/1000,"km")
print(routes['routes'][0]['summary']['duration']/60,"min")

In [13]:
def calculate_itinerary(adresses,city, profile = None):

    coords = geocode_addresses_2(adresses,city)
    routes = client.directions(list(coords.values()), profile=profile, optimize_waypoints=True)

    map_center = [(routes['bbox'][0] + routes['bbox'][2]) / 2, (routes['bbox'][1] + routes['bbox'][3]) / 2]
    geometry=routes['routes'][0]['geometry']
    decoded = convert.decode_polyline(geometry)
    #m= folium.Map(location = map_center,zoom_start=13)
    m= folium.Map(location = list(reversed(map_center)),zoom_start=13)

    for place, coord in coords.items():
        folium.Marker(
            location=list(reversed(coord)),
            popup=f"{place}",
            icon=folium.Icon(color="red", icon="info-sign")
        ).add_to(m)

    total_distance = routes['routes'][0]['summary']['distance']  # en mètres
    total_duration = routes['routes'][0]['summary']['duration']  # en secondes
   
    # Texte dynamique
    distance_km = total_distance / 1000
    duration_minutes = total_duration / 60
    popup_text = (
        f"<b>Total distance:</b> {distance_km:.2f} km<br>"
        f"<b>Total duration:</b> {duration_minutes:.1f} minutes"
    )

    folium.PolyLine(
        locations =  [list(reversed(coord)) for coord in decoded['coordinates']],
        color='blue',
        weight = 5,
        tooltip=popup_text,
        ).add_to(m)

    return m


In [14]:
ville = 'Athènes'
list_act = ["musée de l'acropole", "parthenon","plaka","stade panathenaïque","acropole","stade panathenaïque","mont lycabette","musée archeologique national","temple d'héphaïsteion"]



In [ ]:
m=calculate_itinerary(list_act,ville,'cycling-regular')
m

In [ ]:
address = 'la tour eiffel'
city = 'paris'
response = client.pelias_search(text=f"{address},{city}")

In [ ]:
coordinates = response['features'][0]['geometry']['coordinates']

In [ ]:
list(reversed(coordinates))

In [ ]:
geolocator = Nominatim(user_agent="test")
tab=[]
city="athenes"
for address in list_act:
    location = geolocator.geocode(f"{address},{city}")
    #time.sleep(1)
    if location is not None:
          tab.append([location.latitude,location.longitude])
    else:
         tab.append([None,None])

In [ ]:
location = geolocator.geocode(f"mont lycabette,{city}")

location

In [ ]:
type(location)

In [ ]:
from geopy.geocoders import Nominatim
from geopy.exc import GeopyError

def test(city_name):
    """
    Valide et récupère le nom standardisé d'une ville en anglais via l'API Nominatim d'OpenStreetMap.
    
    Args:
        city_name (str): Le nom de la ville à valider.
        
    Returns:
        str: Le nom standardisé de la ville en anglais, ou None si la ville n'est pas trouvée.
    """
    # Initialiser le géolocalisateur avec un user_agent
    geolocator = Nominatim(user_agent="city_validation_test")
    
    try:
        # Rechercher la ville, en demandant les résultats en anglais
        location = geolocator.geocode(city_name, addressdetails=True, exactly_one=True, language='en')
        
        if location:
            # Extraction du nom de la ville en anglais
            standardized_name = location.address.split(",")[0]
            print(f"City found: {standardized_name}")
            return standardized_name
        else:
            print(f"City '{city_name}' not found.")
            return None
    except GeopyError as e:
        print(f"Error querying city API: {e}")
        return None

# Exemple d'utilisation
test('amien, france')


In [ ]:
from geopy.geocoders import Nominatim
from geopy.exc import GeopyError

def test(city_name, country_name):
    """
    Valide et récupère le nom standardisé d'une ville avec le pays en anglais via l'API Nominatim d'OpenStreetMap.
    Permet d'envoyer à la fois le nom de la ville et du pays pour une recherche plus précise.
    
    Args:
        city_name (str): Le nom de la ville à valider.
        country_name (str): Le nom du pays dans lequel chercher la ville.
        
    Returns:
        str: Le nom standardisé de la ville et du pays en anglais, ou None si la ville n'est pas trouvée.
    """
    # Initialiser le géolocalisateur avec un user_agent
    geolocator = Nominatim(user_agent="city_validation_test")
    
    try:
        # Formater la requête avec la ville et le pays
        query = f"{city_name}, {country_name}"
        
        # Rechercher la ville dans le pays, en demandant les résultats en anglais
        location = geolocator.geocode(query, addressdetails=True, exactly_one=True, language='en')
        
        if location:
            # Extraction du nom de la ville et du pays en anglais
            address_parts = location.raw.get('address', {})

            # On prend la première partie (ville) et la dernière (pays)
            #city_and_country = f"{address_parts[0]}, {address_parts[-1]}"
            address = location.raw.get('address', {})
            region = address.get('state', None)
            city = address.get('city', None)
            country = address.get('country', None)

            #print(f"City found: {city_and_country}")
            return city,country,region
        else:
            print(f"City '{city_name}' in '{country_name}' not found.")
            return test(input("enter a city"),input("enter a country"))
    except GeopyError as e:
        print(f"Error querying city API: {e}")
        

# Exemple d'utilisation
a,b,c = test('montpellier', 'france')
print(a)
print(b)
print(c)
